# Importing Common Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import timedelta
from datetime import time
from datetime import datetime
import pickle

In [6]:
reliance=pd.read_excel('data/Reliance.xlsx')
df=reliance
df=df.set_index('Date')
df.tail(5)

,Date,Open
7113,2021-05-21,1995.70
7114,2021-05-24,2007.40
7115,2021-05-25,1990.05
7116,2021-05-26,1984.90
7117,2021-05-27,1970.90


In [15]:
training_set=df.iloc[:,0:1].values
training_set

array([[ 365.  ],
       [ 330.  ],
       [ 285.  ],
       ...,
       [1990.05],
       [1984.9 ],
       [1970.9 ]])

In [16]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1),copy=True)
training_set_scaled=sc.fit_transform(training_set)
training_set_scaled

array([[0.08374038],
       [0.07256648],
       [0.05820004],
       ...,
       [0.60254446],
       [0.6009003 ],
       [0.59643074]])

### Creating a data structure with 60 timesteps and 1 output

In [17]:
import numpy as np
X_train=[]
y_train=[]
for i in range(60,len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)

### Reshaping

In [18]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Part 2 - Building and Training the RNN

### Importing the Keras libraries and packages

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Initialising the RNN

In [20]:
regressor = Sequential()

### Adding the first LSTM layer and some Dropout regularisation

In [21]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

### Adding a second LSTM layer and some Dropout regularisation

In [22]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a third LSTM layer and some Dropout regularisation

In [23]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a fourth LSTM layer and some Dropout regularisation

In [24]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

### Adding the output layer

In [25]:
regressor.add(Dense(units = 1))

### Compiling the RNN

In [26]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Fitting the RNN to the Training set

In [28]:
regressor.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5
221/221 [==============================] - 31s 142ms/step - loss: 0.0010
Epoch 2/5
221/221 [==============================] - 32s 143ms/step - loss: 8.8466e-04
Epoch 3/5
221/221 [==============================] - 30s 136ms/step - loss: 8.2809e-04
Epoch 4/5
221/221 [==============================] - 32s 145ms/step - loss: 7.8973e-04
Epoch 5/5
221/221 [==============================] - 32s 144ms/step - loss: 7.8227e-04


In [29]:
regressor.save("reliance_model")

INFO:tensorflow:Assets written to: reliance_model\assets


INFO:tensorflow:Assets written to: reliance_model\assets


In [31]:
from tensorflow import keras
model = keras.models.load_model("reliance_model")